In [1]:
import pandas as pd
import numpy as np

PARTE A

In [2]:
# La página se ha caído, se descarga el dataset
# url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"

In [3]:
data = pd.read_csv("breast-cancer-wisconsin.csv")

In [4]:
data.head(5)

,id,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


Se revisan valores faltantes

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 699 non-null    int64 
 1   clump_thickness    699 non-null    int64 
 2   size_uniformity    699 non-null    int64 
 3   shape_uniformity   699 non-null    int64 
 4   marginal_adhesion  699 non-null    int64 
 5   epithelial_size    699 non-null    int64 
 6   bare_nucleoli      699 non-null    object
 7   bland_chromatin    699 non-null    int64 
 8   normal_nucleoli    699 non-null    int64 
 9   mitoses            699 non-null    int64 
 10  class              699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [6]:
data.isnull().sum()

id                   0
clump_thickness      0
size_uniformity      0
shape_uniformity     0
marginal_adhesion    0
epithelial_size      0
bare_nucleoli        0
bland_chromatin      0
normal_nucleoli      0
mitoses              0
class                0
dtype: int64

Se busca que tipo de valores pueden ser datos perdidos

In [7]:
data['bare_nucleoli'].unique()

array(['1', '10', '2', '4', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

Se observa un un valor perdido (?). Se determina la cantidad de datos perdidos

In [8]:
(data['bare_nucleoli'] == '?').sum()

16

Para imputar el dato con la mediana, se requiere convertir la variable a numérica. Se reemplazan los valores "?" por NA

In [9]:
data['bare_nucleoli'] = data['bare_nucleoli'].replace("?",pd.NA)

In [10]:
data.isnull().sum()

id                    0
clump_thickness       0
size_uniformity       0
shape_uniformity      0
marginal_adhesion     0
epithelial_size       0
bare_nucleoli        16
bland_chromatin       0
normal_nucleoli       0
mitoses               0
class                 0
dtype: int64

In [11]:
data['bare_nucleoli'] = pd.to_numeric(data['bare_nucleoli'])

In [12]:
data['bare_nucleoli'].dtype

dtype('float64')

Se calcula la mediana y se reemplaza en los valores faltantes

In [13]:
mediana = data['bare_nucleoli'].median()
data['bare_nucleoli'] = data['bare_nucleoli'].fillna(mediana)

In [14]:
data['bare_nucleoli'] = data['bare_nucleoli'].astype(int)

In [15]:
data['bare_nucleoli'].unique()

array([ 1, 10,  2,  4,  3,  9,  7,  5,  8,  6])

Se revisa la variable 'Class'

In [16]:
data['class'].unique()

array([2, 4], dtype=int64)

Convertir los valores "2" y "4" a "0" y "1"

In [17]:
data['class'] = data['class'].apply(lambda x: 0 if x == 2 else 1)

In [18]:
data['class'].unique()

array([0, 1], dtype=int64)

In [19]:
data['class'] = data['class'].astype('category')

In [20]:
data['id'] = data['id'].astype(str)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 699 non-null    object  
 1   clump_thickness    699 non-null    int64   
 2   size_uniformity    699 non-null    int64   
 3   shape_uniformity   699 non-null    int64   
 4   marginal_adhesion  699 non-null    int64   
 5   epithelial_size    699 non-null    int64   
 6   bare_nucleoli      699 non-null    int32   
 7   bland_chromatin    699 non-null    int64   
 8   normal_nucleoli    699 non-null    int64   
 9   mitoses            699 non-null    int64   
 10  class              699 non-null    category
dtypes: category(1), int32(1), int64(8), object(1)
memory usage: 52.8+ KB


PARTE B

- Detección valores atípicos univariados mediante rango intercuartil

Se define el rango intercuartil:

Q1 = primer cuartil (25% de los datos por debajo)

Q3 = tercer cuartil (75% de los datos por debajo)

IQR = Q3 − Q1 (rango intercuartílico)

Luego se definen los límites para detectar outliers:

Límite inferior = Q1 − (k × IQR)

Límite superior = Q3 + (k × IQR)

Para este caso k=3

Se seleccionan solamente las variables numéricas

In [22]:
var_numeric_ric = data.select_dtypes(include='number').copy()

Se revisan los estadísticos de las variables

In [23]:
var_numeric_ric.describe()

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.486409,3.437768,2.866953,1.589413
std,2.815741,3.051459,2.971913,2.855379,2.214300,3.621929,2.438364,3.053634,1.715078
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


Se observa una distribución atípica en la variable mitoses

Se recorren las columnas para determinar los rangos intercuartiles, detectar los outliers y reemplazar por valores nulos

In [24]:
for col in var_numeric_ric:
    Q1 = var_numeric_ric[col].quantile(0.25)
    Q3 = var_numeric_ric[col].quantile(0.75)
    
    IQR = Q3-Q1
    k = 3
    
    lim_inf = Q1 - k*IQR
    lim_sup = Q3 + k*IQR
    
    # Se crea una máscara booleanda para detectar outliers
    mask_outliers = (var_numeric_ric[col] < lim_inf) | (var_numeric_ric[col] > lim_sup)

    # Se reemplazan los outliers (valores True) por Na
    var_numeric_ric.loc[mask_outliers, col] = np.nan

Se contabilizan los outliers

In [25]:
var_numeric_ric.isna().sum()

clump_thickness        0
size_uniformity        0
shape_uniformity       0
marginal_adhesion      0
epithelial_size        0
bare_nucleoli          0
bland_chromatin        0
normal_nucleoli        0
mitoses              120
dtype: int64

Se observa que el 17% de los valores de la variable 'mitoses' son atípicos. Podría considerarse eliminar la variable

- Detección de valores atípicos multivariados mediante Mahalanobis

In [26]:
from scipy.spatial.distance import mahalanobis

Los valores del dataframe se trabajan como array para una mayor eficiencia

In [27]:
var_numeric_maha = data.select_dtypes(include='number').copy()

In [28]:
X = var_numeric_maha.values

Se calcula la media multivariada

In [34]:
mean_vec = np.mean(X, axis = 0)

In [35]:
mean_vec

array([4.41773963, 3.13447783, 3.2074392 , 2.80686695, 3.21602289,
       3.48640916, 3.43776824, 2.86695279, 1.58941345])

Se calcula la matriz de covarianza inversa

In [32]:
cov_matrix = np.cov(X, rowvar=False)
inv_cov_matrix = np.linalg.inv(cov_matrix)

In [33]:
inv_cov_matrix

array([[ 0.24136756, -0.04326361, -0.05779416,  0.01987167, -0.00115685,
        -0.04445912, -0.01626922, -0.01403337, -0.0217344 ],
       [-0.04326361,  0.77231609, -0.47065263, -0.06917098, -0.15416152,
         0.01257058, -0.11311914, -0.03938676, -0.02772149],
       [-0.05779416, -0.47065263,  0.73564684, -0.00851994, -0.04052871,
        -0.06486148, -0.03090451, -0.06220755, -0.00172577],
       [ 0.01987167, -0.06917098, -0.00851994,  0.3016256 , -0.02057304,
        -0.06644805, -0.06712287, -0.0222236 , -0.04764495],
       [-0.00115685, -0.15416152, -0.04052871, -0.02057304,  0.51906263,
        -0.01885214, -0.01533556, -0.04030569, -0.09759057],
       [-0.04445912,  0.01257058, -0.06486148, -0.06644805, -0.01885214,
         0.19344258, -0.06082612,  0.00229545,  0.01062166],
       [-0.01626922, -0.11311914, -0.03090451, -0.06712287, -0.01533556,
        -0.06082612,  0.48270562, -0.06897697,  0.04171983],
       [-0.01403337, -0.03938676, -0.06220755, -0.0222236 , -0

Cacular la distancia de Mahalanobis para cada fila

In [37]:
mahal_dist = [mahalanobis(row, mean, inv_cov_matrix) for row in X]

Se agregar la lista al dataframe

In [68]:
var_numeric_maha['mahal_dist'] = mahal_dist

In [69]:
var_numeric_maha.head()

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses,mahal_dist
0,5,1,1,1,2,1,3,1,1,1.462433
1,5,4,4,5,7,10,3,2,1,3.736788
2,3,1,1,1,2,2,3,1,1,1.040197
3,6,8,8,1,3,4,3,7,1,4.098292
4,4,1,1,3,2,1,3,1,1,1.493547


Se eliminan aquellas filas cuyo valor sea mayor a 30

In [70]:
var_numeric_filt = var_numeric_maha[var_numeric_maha['mahal_dist']<=30]

In [71]:
var_numeric_filt.shape

(699, 10)

Se ha mantenido el tamaño del dataframe, por lo que no se han detectado valores atípicos multivariados

PARTE C

Consideranvo la variable clase como la variable independiente, y el resto de variables numéricas como las predictoras, se aplica la transformación
sobre los conjuntos de datos

- Transformación Z-score

In [72]:
var_numeric = data.select_dtypes(include = 'number')

In [73]:
var_numeric_z = (var_numeric - var_numeric.mean()) / var_numeric.std()

In [74]:
var_numeric_z.head()

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses
0,0.206788,-0.699494,-0.742767,-0.632794,-0.549168,-0.686488,-0.179534,-0.611387,-0.343666
1,0.206788,0.283642,0.266684,0.768071,1.708882,1.798376,-0.179534,-0.283909,-0.343666
2,-0.503505,-0.699494,-0.742767,-0.632794,-0.549168,-0.410392,-0.179534,-0.611387,-0.343666
3,0.561934,1.594490,1.612618,-0.632794,-0.097558,0.141800,-0.179534,1.353485,-0.343666
4,-0.148359,-0.699494,-0.742767,0.067638,-0.549168,-0.686488,-0.179534,-0.611387,-0.343666


- Transformación de normalización Min - Max

In [76]:
var_numeric_min_max = (var_numeric - var_numeric.mean())/(var_numeric.max()-var_numeric.min())

In [77]:
var_numeric_min_max.head()

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses
0,0.064696,-0.237164,-0.245271,-0.200763,-0.135114,-0.276268,-0.048641,-0.207439,-0.06549
1,0.064696,0.096169,0.088062,0.243681,0.420442,0.723732,-0.048641,-0.096328,-0.06549
2,-0.157527,-0.237164,-0.245271,-0.200763,-0.135114,-0.165157,-0.048641,-0.207439,-0.06549
3,0.175807,0.540614,0.532507,-0.200763,-0.024003,0.057066,-0.048641,0.459227,-0.06549
4,-0.046416,-0.237164,-0.245271,0.021459,-0.135114,-0.276268,-0.048641,-0.207439,-0.06549
